# Retrieval Augmented Generation

## Setup API clients

In [1]:
import os

import azure.identity
import dotenv
import openai
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

dotenv.load_dotenv()

azure_credential = azure.identity.DefaultAzureCredential()

# Initialize Azure OpenAI client
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_ADA_DEPLOYMENT = os.getenv("AZURE_OPENAI_ADA_DEPLOYMENT")

token_provider = azure.identity.get_bearer_token_provider(azure_credential, "https://cognitiveservices.azure.com/.default")
openai_client = openai.AzureOpenAI(
    api_version="2023-07-01-preview",
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com",
    azure_ad_token_provider=token_provider)

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_ADA_DEPLOYMENT, input=text)
    return get_embeddings_response.data[0].embedding

# Initialize Azure search client
AZURE_SEARCH_SERVICE = os.getenv("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_ENDPOINT = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"

AZURE_SEARCH_FULL_INDEX = "gptkbindex"
search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_FULL_INDEX, credential=azure_credential)


## Prepare user question

In [2]:
user_question = "What does a product manager do?"
user_question_vector = get_embedding(user_question)

## Retrieve matching documents

The search call below does a **hybrid search**, performing both a full-text search and a vector search in parallel.
It merges those results using Reciprocal Rank Fusion (RRF). 
Finally, it re-ranks the merged results using the AI Search semantic ranker, a re-ranking model.

In [3]:
r = search_client.search(
        user_question,
        top=5, 
        vector_queries=[
                VectorizedQuery(vector=user_question_vector, k_nearest_neighbors=50, fields="embedding")],
        query_type="semantic",
        semantic_configuration_name="default")

sources = "\n\n".join([f"[{doc['sourcepage']}]: {doc['content']}\n" for doc in r])

print(sources)

[role_library.pdf#page=29]:  The Manager of Product Management will collaborate with internal teams, such as engineering, sales, marketing, and finance, as well as external partners, suppliers, and customers to ensure successful product execution.
Responsibilities:
· Lead the product management team and provide guidance on product strategy, design, development, and launch.
· Develop and implement product life-cycle management processes.
· Monitor and analyze industry trends to identify opportunities for new products.
· Develop product marketing plans and go-to-market strategies.
· Research customer needs and develop customer-centric product roadmaps.
· Collaborate with internal teams to ensure product execution and successful launch.
· Develop pricing strategies and cost models.
· Oversee product portfolio and performance metrics.
· Manage product development budget.
· Analyze product performance and customer feedback to identify areas for improvement.
Qualifications:
· Bachelor's degr

## Send question and documents to the model

In [4]:
SYSTEM_MESSAGE = """
Assistant helps company employees questions about the employee handbook. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below.
Each source has a name followed by colon and the actual information, include the source name for each fact you use.
Use square brackets to reference the source, for example [info1.txt].
"""
USER_MESSAGE = user_question + "\nSources: " + sources

# Now we can use the matches to generate a response
response = openai_client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    temperature=0.7,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

answer = response.choices[0].message.content
print(answer)

A product manager is responsible for leading the product management team and providing guidance on product strategy, design, development, and launch. They collaborate with internal teams and external partners to ensure successful product execution [role_library.pdf#page=29]. They are also responsible for developing and implementing product life-cycle management processes, monitoring industry trends, developing product marketing plans, researching customer needs, and overseeing product portfolio and performance metrics [role_library.pdf#page=29]. They should have a bachelor's degree in business, engineering, or a related field, and at least 5 years of experience in product management with a proven track record of successful product launches [role_library.pdf#page=29].
